<a href="https://colab.research.google.com/github/npar9999/intro-to-dl/blob/master/document_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##
1. tried different models (faster RCNN ) but result was not satisfactory . Based on different articles and papers , retinanet - gives better / faster result 
2. Below is keras implementation of retinanet

#Setup Details 

In [ ]:
import os

In [ ]:
#change Directory to working folder 
os.chdir("/content/drive/My Drive/AIML/")

In [ ]:
# git clone retinanet implementation by fizyr / uncomment 
#!git clone https://github.com/fizyr/keras-retinanet.git

In [ ]:
# change drive to retinatent folder
%cd keras-retinanet/

In [ ]:
# install dependencies /uncomment 
#!pip install .

In [ ]:
# install set up extention /uncomment
#!python setup.py build_ext --inplace

In [ ]:
!pip install keras-resnet

# import packages to be used 

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [ ]:
import keras_resnet

## directory path of images and csv path

In [ ]:
# define path
image_dir = "/content/drive/My Drive/concord/principal_engineer_ai/keras-retinanet/data/images/"
csv_path = "/content/drive/My Drive/concord/principal_engineer_ai/keras-retinanet/data/"


In [ ]:
# CSV dataframe 
df_train = pd.read_csv(csv_path+"train.csv")
df_train.head()

In [ ]:
# rename filds for better understanding 
df_train["FileName"] = df_train["FileName"].apply((lambda x:image_dir+x))
df_train.rename(columns={"FileName": "image_name", "x1": "x_min","y1":"y_min","x2":"x_max","y2":"y_max","class":"class_name"},inplace=True)

In [ ]:
# function to show images 
def show_image_objects(image_row):

  img_path = image_row.image_name
  box = [
    image_row.x_min, image_row.y_min, image_row.x_max, image_row.y_max
  ]

  image = read_image_bgr(img_path) # read image , calls cv2 package 

  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  draw_box(draw, box, color=(0,255, 0)) # show in green color

  plt.axis('off')
  plt.imshow(draw)
  plt.show()

In [ ]:
# check sample image
show_image_objects(df_train.iloc[2])

In [ ]:
# create test data frame for test csv file 
df_test = pd.read_csv(csv_path+"val.csv")

df_test["FileName"] = df_test["FileName"].apply((lambda x:image_dir+x))
df_test.rename(columns={"FileName": "image_name", "x1": "x_min","y1":"y_min","x2":"x_max","y2":"y_max","class":"class_name"},inplace=True)
df_test.head()

In [ ]:
# to pass data to retina net model we need to pass annotation and classes file 
ANNOTATIONS_FILE = 'annotations.csv'
CLASSES_FILE = 'classes.csv'

In [ ]:
# check current directory location
!pwd

In [ ]:
# create annotation file 
df_train.to_csv(ANNOTATIONS_FILE, index=False, header=None)

In [ ]:
# create a class file . In this case since we are looking only at 1 type of image / table is passed as class
classes = set(['table'])

with open(CLASSES_FILE, 'w') as f:
  for i, line in enumerate(sorted(classes)):
    f.write('{},{}\n'.format(line,i))

In [ ]:
!head classes.csv

In [ ]:
!head annotations.csv

In [ ]:
# create directory to store model checkpoints 
os.makedirs("snapshots", exist_ok=True)

In [ ]:

PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'

In [ ]:
# download pretrained model
URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

In [ ]:
#!pwd

In [ ]:
# details for  model invocation 
'''
call train.py
default backbone : resnet50 

parms passed :
freeze-backbone : would disable learning for the backbone 
random-transform : creates random transforms for augmenting train data 

weights  : Initialize the model with weights / pretrained_model
batch-size : 8  (size of batches )
steps : 10 
epochs :  Number of epochs to train 

input format : csv
1.annotations.csv -> csv file consisting of path and annotations 
2.classes.csv
'''

'''
To resume training from snapshots 
pass --snapshots 
-- initial-epochs  (type = int , default is 0)
'''

'''
compilation used 
 # compile model
    training_model.compile(
        loss={
            'regression'    : losses.smooth_l1(),
            'classification': losses.focal()
        },
        optimizer=keras.optimizers.Adam(lr=lr, clipnorm=0.001)
    )

'''

'''
Returns 
model saved in snapshots 
'''

# other parameters taken as default
#lr = 1e-5


# this is stripped down version of model and has values only necessary for training 
#classification and regression values

In [ ]:
!python keras_retinanet/bin/train.py --freeze-backbone --random-transform --weights {PRETRAINED_MODEL} --batch-size 8 --steps 10 --epochs 10 csv annotations.csv classes.csv

#loading trained model 

In [ ]:
!ls snapshots

# training models run on stripped version
# run inference model for detection 

In [ ]:
# to do object detection , we need to convert the model using #convert_model

In [ ]:
model_path = os.path.join('snapshots', sorted(os.listdir('snapshots'), reverse=True)[0])
print(model_path)

model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

labels_to_names = pd.read_csv(CLASSES_FILE, header=None).T.loc[0].to_dict()

# predictions 

In [ ]:
# function to predict image 
def predict(image):
  image = preprocess_image(image.copy()) # preprocess the image 
  image, scale = resize_image(image) # resize image 

# use model to predict bonding boxes / scores and labels 
  boxes, scores, labels = model.predict_on_batch(
    np.expand_dims(image, axis=0))   

  boxes /= scale

  return boxes, scores, labels

In [ ]:
# set threshold score for confidence 
THRES_SCORE = 0.5

# function to draw predicted boxes 
def draw_detections(image, boxes, scores, labels):
  for box, score, label in zip(boxes[0], scores[0], labels[0]):
    if score < THRES_SCORE:
        break

    color = label_color(label)

    b = box.astype(int)
    draw_box(image, b, color=color)

    caption = "{} {:.3f}".format(labels_to_names[label], score)
    draw_caption(image, b, caption)


In [ ]:
# function to show detected objects 
def show_detected_objects(image_row):
  img_path = image_row.image_name
  
  image = read_image_bgr(img_path)
  # call the model and predict (defined above 
  boxes, scores, labels = predict(image)

  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  true_box = [
    image_row.x_min, image_row.y_min, image_row.x_max, image_row.y_max
  ]
  #defined above 
  draw_box(draw, true_box, color=(255,0, 0))
  # defined above
  draw_detections(draw, boxes, scores, labels)

  plt.axis('off')
  plt.imshow(draw)
  plt.show()

In [ ]:
# show sample test data 
df_test.head(n=10)

In [ ]:
# show image object / orignal ground truth 
show_image_objects(df_test.iloc[4])

In [ ]:
# show detected object / predicted value for sample image 
show_detected_objects(df_test.iloc[4])

In [ ]:
$ jupyter nbconvert --to html mynotebook.ipynb